<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/PatternMining/Process_for_structure_pm(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mise en forme des donnees pour le *Pattern Mining*

### Imports

In [0]:
import pandas as pd  
import numpy as np
import warnings

### Recuperation et preparation des donnnees

In [87]:
#Chargement du jeu de donnees
data_child = pd.read_csv('data_madeleine_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col = False)

# Supression de la colonne qui duplique l'index
data_child = data_child.drop(columns = 'Unnamed: 0') 

# Recuperation de la partie phonetique des intervention de l'enfant
data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])

# Conservation des deux colonnes avec des informations pertinantes 
data_child = data_child[['age','contenu']]

# Creer un indexage a partir de cet ordre
data_child['index']=[ i for i in range(len(data_child))]
data_child= data_child.set_index('index')

# Creation d'un numpy utile par la suite pour les boucles
ages = data_child.age.unique()

# Indexage par l'age de l'enfant
age = data_child['age']

l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))

index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
data_child.set_index(index,inplace=True)

# Supression de la colonne age car present en index
data_child=data_child[['contenu']]

data_child.head()

contenu
age     index        
1_00_05 0          eː
        1          bɛ
        2        vœga
        3       gatœ̃
        4        kaka

### Demarches pour la division de chaque enregistrement en 4 parties

In [0]:
# Sépare en mots au lieu de phrases
mots = {}
for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    
# se parcourt avec un age en parametre
#mots["1_00_05"]

In [89]:
# Sépare en 4 "enfants" chaque enregistrement
enregistrement = {}
sizes = []
for i in list(ages):
    # recupere tout les mots a la i eme date
    #data = data_child['contenu'].loc[i].values
    data = mots[i]
    # separe les mots en 4 groupes ( 3 goupes de meme dim et le dernier avec le reste)
    if len(data)<4:
        warnings.warn("\nAttention un l'enregistrement "+ i + " contient moins de 4 mots")
    s = len(data)//4
    sizes.append(s)
    enfant1 = data[:s]
    enfant2 = data[s:2*s]
    enfant3 = data[2*s:3*s]
    enfant4 = data[3*s:]
    # creer une variable texte de chaque goupe
    #enfant1 = ' '.join(enfant1)
    #enfant2 = ' '.join(enfant2)
    #enfant3 = ' '.join(enfant3)
    #enfant4 = ' '.join(enfant4)
    
    enfant1 = tuple(enfant1)
    enfant2 = tuple(enfant2)
    enfant3 = tuple(enfant3)
    enfant4 = tuple(enfant4)
    
    # sépare en 4 enfants
    enregistrement[i] = [enfant1, enfant2, enfant3, enfant4]
    
# se parcourt avec un age en parametre
#enregistrement["1_00_05"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: 
Attention un l'enregistrement 6_11_27 contient moins de 4 mots
  if __name__ == '__main__':


In [90]:
# Pour avoir une idée du nombre de mots par enregistrement après répartition
pd.DataFrame(sizes[:-1]).describe().T

,count,mean,std,min,25%,50%,75%,max
0,34.0,395.588235,285.178744,15.0,126.75,364.0,646.25,1101.0


In [91]:
# Crée une DataFrame de toutes les donnees
tab_4groupes = []
#sizes = []
for i in list(ages):
    t = enregistrement[i]
    #s = len(t)
    tab_4groupes.append(t)
    #sizes.append(s)
    
tab_madeleine = pd.DataFrame(tab_4groupes)

etiquettes = ["madeleine_enfant1",
              "madeleine_enfant2", 
              "madeleine_enfant3", 
              "madeleine_enfant4"]

tab_madeleine.columns = etiquettes

tab_madeleine.head()
# tab_madeleine.loc[1] recherche par numero de l'enregistrement
# tab_madeleine[1] recherche par numero de l'enfant

,madeleine_enfant1,madeleine_enfant2,madeleine_enfant3,madeleine_enfant4
0,"(eː, bɛ, vœga, gatœ̃, kaka, dœjøkɛka, m, taka,...","(gla, ga, ga, gøga, gøga, ga, gaga, a, deda, œ...","(gaga, gaga, gagaga, ga, mœmœmœmœ, ala, adla, ...","(œmøː, œmøː, gaga, gagaga, daɥy, aga, œm, œm, ..."
1,"(ja, aljɛla, jaka, meja, agø, ga, beːja, ʃa, o...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, aja, ja, ...","(ʃa, ejaː, edjadjajaaa, edahahaoga, u, ajne, e...","(ada, aja, gɛʒeʒa, ajəda, dedaː, da, ega, ja, ..."
2,"(je, ʃidɔm, øʃi, alapa, ʃː, ʃ, a, ʃa, øː, aaː,...","(bɛ, y, øː, ʃ, mememebuja, eaa, iɲa, ia, ʃɛ̃, ...","(ʃa, ʒy, aːaːɛ, ɛɛ, sə, wauːua, a, χ, wa, waa,...","(bidun, bida, bjɛtna, bebøː, dakula, daguba, m..."
3,"(evøpuʃa, gegedɔ̃, dɔ, ty, hɛsys, ejə, ʃa, gøt...","(mama, mama, bekaliʃaʃa, øtatɛ, mama, løʃa, pe...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃːa, səba, ʃa, ʃa...","(lelɛ, lala, lala, uu, iː, jejejejejee, eː, mː..."
4,"(i, lɛ, vwa, a, aː, ti, vwa, aː, ty, pø, t, as...","(wikɔ, ejdøwa, ɛʃa, pøpa, ɛøgødididitetɛe, bøb...","(wœ, wa, wawa, pwapwa, awœ, œœ, ba, m, jeje, e...","(jije, ini, œː, i, ʃi, uba, veve, tilele, nine..."


### Addaptation de la structure

In [0]:
# transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
# doit etre de cette forme:
# madeleine_enfant1 = transactions
# un enregistrement = ('a', 'b', 'c')
# un mot = 'a'

In [0]:
madeleine_enfant1 = tuple(tab_madeleine["madeleine_enfant1"][:-1])
madeleine_enfant2 = tuple(tab_madeleine["madeleine_enfant2"][:-1])
madeleine_enfant3 = tuple(tab_madeleine["madeleine_enfant3"][:-1])
madeleine_enfant4 = tuple(tab_madeleine["madeleine_enfant4"][:])

In [94]:
print(type(madeleine_enfant1))
#print(madeleine_enfant1)

<class 'tuple'>


### PATTERN MINING

In [95]:
!pip install pymining
from pymining import itemmining
from pymining import assocrules
from pymining import seqmining

### support min = 20

In [0]:
min_supp = 25
relim_input_1 = itemmining.get_relim_input(madeleine_enfant1)
relim_input_2 = itemmining.get_relim_input(madeleine_enfant2)
relim_input_3 = itemmining.get_relim_input(madeleine_enfant3)
relim_input_4 = itemmining.get_relim_input(madeleine_enfant4)

item_sets_1 = itemmining.relim(relim_input_1, min_support=min_supp) 
item_sets_2 = itemmining.relim(relim_input_2, min_support=min_supp)
item_sets_3 = itemmining.relim(relim_input_3, min_support=min_supp)
item_sets_4 = itemmining.relim(relim_input_4, min_support=min_supp)

#item_sets

In [97]:
print(len(item_sets_1))
print(len(item_sets_2))
print(len(item_sets_3))
print(len(item_sets_4))

20
20
14
18


In [98]:
item_sets_1

{frozenset({'l'}): 25,
 frozenset({'a', 'l'}): 25,
 frozenset({'pa'}): 25,
 frozenset({'la', 'pa'}): 25,
 frozenset({'a'}): 33,
 frozenset({'a', 'pa'}): 25,
 frozenset({'a', 'la', 'pa'}): 25,
 frozenset({'sa'}): 25,
 frozenset({'la', 'sa'}): 25,
 frozenset({'a', 'sa'}): 25,
 frozenset({'a', 'la', 'sa'}): 25,
 frozenset({'ɛ'}): 25,
 frozenset({'m'}): 26,
 frozenset({'a', 'm'}): 26,
 frozenset({'la'}): 29,
 frozenset({'e', 'la'}): 26,
 frozenset({'a', 'la'}): 29,
 frozenset({'e'}): 30,
 frozenset({'a', 'e'}): 29,
 frozenset({'a', 'e', 'la'}): 26}

In [99]:
item_sets_2

{frozenset({'lə'}): 25,
 frozenset({'m'}): 25,
 frozenset({'a', 'm'}): 25,
 frozenset({'X'}): 26,
 frozenset({'X', 'a'}): 25,
 frozenset({'e'}): 28,
 frozenset({'la'}): 28,
 frozenset({'e', 'la'}): 26,
 frozenset({'sa'}): 28,
 frozenset({'e', 'la', 'sa'}): 26,
 frozenset({'a', 'e', 'la'}): 26,
 frozenset({'e', 'sa'}): 27,
 frozenset({'a', 'e'}): 28,
 frozenset({'a', 'e', 'sa'}): 27,
 frozenset({'la', 'sa'}): 27,
 frozenset({'a'}): 32,
 frozenset({'a', 'la'}): 28,
 frozenset({'a', 'sa'}): 28,
 frozenset({'a', 'la', 'sa'}): 27,
 frozenset({'a', 'e', 'la', 'sa'}): 26}

In [100]:
item_sets_3

{frozenset({'sa'}): 25,
 frozenset({'e', 'sa'}): 25,
 frozenset({'a', 'sa'}): 25,
 frozenset({'a', 'e', 'sa'}): 25,
 frozenset({'i'}): 26,
 frozenset({'la'}): 26,
 frozenset({'i', 'la'}): 25,
 frozenset({'e'}): 30,
 frozenset({'e', 'i'}): 25,
 frozenset({'e', 'la'}): 25,
 frozenset({'a'}): 31,
 frozenset({'a', 'la'}): 25,
 frozenset({'a', 'e'}): 29,
 frozenset({'a', 'e', 'la'}): 25}

In [101]:
item_sets_4

{frozenset({'sa'}): 26,
 frozenset({'la', 'sa'}): 25,
 frozenset({'e'}): 30,
 frozenset({'e', 'sa'}): 26,
 frozenset({'e', 'la', 'sa'}): 25,
 frozenset({'a'}): 32,
 frozenset({'a', 'sa'}): 25,
 frozenset({'a', 'la'}): 27,
 frozenset({'a', 'la', 'sa'}): 25,
 frozenset({'a', 'e'}): 28,
 frozenset({'a', 'e', 'sa'}): 25,
 frozenset({'la'}): 27,
 frozenset({'e', 'la'}): 26,
 frozenset({'a', 'e', 'la'}): 26,
 frozenset({'a', 'e', 'la', 'sa'}): 25,
 frozenset({'mamɑ̃'}): 27,
 frozenset({'e', 'mamɑ̃'}): 25,
 frozenset({'a', 'mamɑ̃'}): 26}

In [0]:
rules_1 = assocrules.mine_assoc_rules(item_sets_1, min_support=min_supp, min_confidence=0.8)
rules_2 = assocrules.mine_assoc_rules(item_sets_2, min_support=min_supp, min_confidence=0.8)
rules_3 = assocrules.mine_assoc_rules(item_sets_3, min_support=min_supp, min_confidence=0.8)
rules_4 = assocrules.mine_assoc_rules(item_sets_4, min_support=min_supp, min_confidence=0.8)

In [103]:
print(len(rules_1))
print(len(rules_2))
print(len(rules_3))
print(len(rules_4))

29
52
26
50


In [104]:
rules_1

[(frozenset({'a', 'pa'}), frozenset({'la'}), 25, 1.0),
 (frozenset({'pa'}), frozenset({'a', 'la'}), 25, 1.0),
 (frozenset({'la', 'pa'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'la'}), frozenset({'a', 'pa'}), 25, 0.8620689655172413),
 (frozenset({'a', 'la'}), frozenset({'pa'}), 25, 0.8620689655172413),
 (frozenset({'a', 'sa'}), frozenset({'la'}), 25, 1.0),
 (frozenset({'sa'}), frozenset({'a', 'la'}), 25, 1.0),
 (frozenset({'la', 'sa'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'la'}), frozenset({'a', 'sa'}), 25, 0.8620689655172413),
 (frozenset({'a', 'la'}), frozenset({'sa'}), 25, 0.8620689655172413),
 (frozenset({'a', 'e'}), frozenset({'la'}), 26, 0.896551724137931),
 (frozenset({'e'}), frozenset({'a', 'la'}), 26, 0.8666666666666667),
 (frozenset({'e', 'la'}), frozenset({'a'}), 26, 1.0),
 (frozenset({'la'}), frozenset({'a', 'e'}), 26, 0.896551724137931),
 (frozenset({'a', 'la'}), frozenset({'e'}), 26, 0.896551724137931),
 (frozenset({'l'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'p

In [105]:
rules_2

[(frozenset({'a', 'e', 'la'}), frozenset({'sa'}), 26, 1.0),
 (frozenset({'a', 'e'}), frozenset({'la', 'sa'}), 26, 0.9285714285714286),
 (frozenset({'e'}), frozenset({'a', 'la', 'sa'}), 26, 0.9285714285714286),
 (frozenset({'a'}), frozenset({'e', 'la', 'sa'}), 26, 0.8125),
 (frozenset({'e', 'la'}), frozenset({'a', 'sa'}), 26, 1.0),
 (frozenset({'la'}), frozenset({'a', 'e', 'sa'}), 26, 0.9285714285714286),
 (frozenset({'a', 'la'}), frozenset({'e', 'sa'}), 26, 0.9285714285714286),
 (frozenset({'a', 'e', 'sa'}), frozenset({'la'}), 26, 0.9629629629629629),
 (frozenset({'e', 'sa'}), frozenset({'a', 'la'}), 26, 0.9629629629629629),
 (frozenset({'sa'}), frozenset({'a', 'e', 'la'}), 26, 0.9285714285714286),
 (frozenset({'a', 'sa'}), frozenset({'e', 'la'}), 26, 0.9285714285714286),
 (frozenset({'e', 'la', 'sa'}), frozenset({'a'}), 26, 1.0),
 (frozenset({'la', 'sa'}), frozenset({'a', 'e'}), 26, 0.9629629629629629),
 (frozenset({'a', 'la', 'sa'}), frozenset({'e'}), 26, 0.9629629629629629),
 (froze

In [106]:
rules_3

[(frozenset({'a', 'e'}), frozenset({'sa'}), 25, 0.8620689655172413),
 (frozenset({'e'}), frozenset({'a', 'sa'}), 25, 0.8333333333333334),
 (frozenset({'a'}), frozenset({'e', 'sa'}), 25, 0.8064516129032258),
 (frozenset({'e', 'sa'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'sa'}), frozenset({'a', 'e'}), 25, 1.0),
 (frozenset({'a', 'sa'}), frozenset({'e'}), 25, 1.0),
 (frozenset({'a', 'e'}), frozenset({'la'}), 25, 0.8620689655172413),
 (frozenset({'e'}), frozenset({'a', 'la'}), 25, 0.8333333333333334),
 (frozenset({'a'}), frozenset({'e', 'la'}), 25, 0.8064516129032258),
 (frozenset({'e', 'la'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'la'}), frozenset({'a', 'e'}), 25, 0.9615384615384616),
 (frozenset({'a', 'la'}), frozenset({'e'}), 25, 1.0),
 (frozenset({'e'}), frozenset({'sa'}), 25, 0.8333333333333334),
 (frozenset({'sa'}), frozenset({'e'}), 25, 1.0),
 (frozenset({'a'}), frozenset({'sa'}), 25, 0.8064516129032258),
 (frozenset({'sa'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'i'}), f

In [107]:
rules_4

[(frozenset({'a', 'e', 'sa'}), frozenset({'la'}), 25, 1.0),
 (frozenset({'a', 'e'}), frozenset({'la', 'sa'}), 25, 0.8928571428571429),
 (frozenset({'e'}), frozenset({'a', 'la', 'sa'}), 25, 0.8333333333333334),
 (frozenset({'e', 'sa'}), frozenset({'a', 'la'}), 25, 0.9615384615384616),
 (frozenset({'sa'}), frozenset({'a', 'e', 'la'}), 25, 0.9615384615384616),
 (frozenset({'a', 'sa'}), frozenset({'e', 'la'}), 25, 1.0),
 (frozenset({'a', 'e', 'la'}), frozenset({'sa'}), 25, 0.9615384615384616),
 (frozenset({'e', 'la'}), frozenset({'a', 'sa'}), 25, 0.9615384615384616),
 (frozenset({'la'}), frozenset({'a', 'e', 'sa'}), 25, 0.9259259259259259),
 (frozenset({'a', 'la'}), frozenset({'e', 'sa'}), 25, 0.9259259259259259),
 (frozenset({'e', 'la', 'sa'}), frozenset({'a'}), 25, 1.0),
 (frozenset({'la', 'sa'}), frozenset({'a', 'e'}), 25, 1.0),
 (frozenset({'a', 'la', 'sa'}), frozenset({'e'}), 25, 1.0),
 (frozenset({'e', 'sa'}), frozenset({'la'}), 25, 0.9615384615384616),
 (frozenset({'e'}), frozenset(

In [0]:
seqs_1 = madeleine_enfant1
seqs_2 = madeleine_enfant2
seqs_3 = madeleine_enfant3
seqs_4 = madeleine_enfant4

freq_seqs_1 = seqmining.freq_seq_enum(seqs_1, 20)
freq_seqs_2 = seqmining.freq_seq_enum(seqs_2, 20)
freq_seqs_3 = seqmining.freq_seq_enum(seqs_3, 20)
freq_seqs_4 = seqmining.freq_seq_enum(seqs_4, 20)

#for seq in freq_seqs:
#    print (seq)

In [109]:
print(len(freq_seqs_1))
print(len(freq_seqs_2))
print(len(freq_seqs_3))
print(len(freq_seqs_4))

4800
5373
10290
5640


In [110]:
print(freq_seqs_1)

{(('la', 'a', 'la', 'la', 'la', 'la', 'a'), 20), (('a', 'a', 'le', 'la', 'a'), 21), (('a', 'e', 'a', 'e', 'le', 'e'), 21), (('e', 'e', 'a', 'e', 'le'), 21), (('e', 'la', 'a', 'e', 'a', 'a', 'a', 'a'), 20), (('a', 'e', 'le', 'la', 'a'), 21), (('a', 'sa', 'i', 'le', 'la'), 20), (('e', 'a', 'sa', 'a', 'le', 'a'), 20), (('lə', 'a', 'a'), 20), (('e', 'sa', 'la', 'sa'), 21), (('la', 'e', 'a', 'le'), 21), (('pa', 'a', 'sa', 'la', 'le', 'a'), 20), (('a', 'la', 'e', 'la', 'i'), 20), (('e', 'e', 'e', 'sa', 'le', 'a'), 20), (('e', 'pa', 'a', 'sa', 'la', 'la'), 20), (('a', 'a', 'e', 'la', 'la', 'a'), 20), (('pa', 'la', 'a', 'la', 'le'), 20), (('a', 'sa', 'la', 'e', 'e', 'e'), 20), (('sa', 'e', 'le', 'a'), 21), (('e', 'a', 'la', 'e', 'sa'), 21), (('e', 'sa', 'la', 'pa', 'a'), 21), (('pa', 'a', 'sa', 'a', 'a'), 20), (('sa', 'a', 'pa', 'a'), 20), (('a', 'i', 'la', 'le'), 20), (('la', 'la', 'a', 'la', 'a', 'a'), 20), (('la', 'la', 'a', 'i', 'a'), 21), (('e', 'e', 'la', 'a', 'a', 'a', 'e'), 20), (('e',

In [111]:
print(freq_seqs_1)

{(('la', 'a', 'la', 'la', 'la', 'la', 'a'), 20), (('a', 'a', 'le', 'la', 'a'), 21), (('a', 'e', 'a', 'e', 'le', 'e'), 21), (('e', 'e', 'a', 'e', 'le'), 21), (('e', 'la', 'a', 'e', 'a', 'a', 'a', 'a'), 20), (('a', 'e', 'le', 'la', 'a'), 21), (('a', 'sa', 'i', 'le', 'la'), 20), (('e', 'a', 'sa', 'a', 'le', 'a'), 20), (('lə', 'a', 'a'), 20), (('e', 'sa', 'la', 'sa'), 21), (('la', 'e', 'a', 'le'), 21), (('pa', 'a', 'sa', 'la', 'le', 'a'), 20), (('a', 'la', 'e', 'la', 'i'), 20), (('e', 'e', 'e', 'sa', 'le', 'a'), 20), (('e', 'pa', 'a', 'sa', 'la', 'la'), 20), (('a', 'a', 'e', 'la', 'la', 'a'), 20), (('pa', 'la', 'a', 'la', 'le'), 20), (('a', 'sa', 'la', 'e', 'e', 'e'), 20), (('sa', 'e', 'le', 'a'), 21), (('e', 'a', 'la', 'e', 'sa'), 21), (('e', 'sa', 'la', 'pa', 'a'), 21), (('pa', 'a', 'sa', 'a', 'a'), 20), (('sa', 'a', 'pa', 'a'), 20), (('a', 'i', 'la', 'le'), 20), (('la', 'la', 'a', 'la', 'a', 'a'), 20), (('la', 'la', 'a', 'i', 'a'), 21), (('e', 'e', 'la', 'a', 'a', 'a', 'e'), 20), (('e',

In [112]:
print(freq_seqs_1)

{(('la', 'a', 'la', 'la', 'la', 'la', 'a'), 20), (('a', 'a', 'le', 'la', 'a'), 21), (('a', 'e', 'a', 'e', 'le', 'e'), 21), (('e', 'e', 'a', 'e', 'le'), 21), (('e', 'la', 'a', 'e', 'a', 'a', 'a', 'a'), 20), (('a', 'e', 'le', 'la', 'a'), 21), (('a', 'sa', 'i', 'le', 'la'), 20), (('e', 'a', 'sa', 'a', 'le', 'a'), 20), (('lə', 'a', 'a'), 20), (('e', 'sa', 'la', 'sa'), 21), (('la', 'e', 'a', 'le'), 21), (('pa', 'a', 'sa', 'la', 'le', 'a'), 20), (('a', 'la', 'e', 'la', 'i'), 20), (('e', 'e', 'e', 'sa', 'le', 'a'), 20), (('e', 'pa', 'a', 'sa', 'la', 'la'), 20), (('a', 'a', 'e', 'la', 'la', 'a'), 20), (('pa', 'la', 'a', 'la', 'le'), 20), (('a', 'sa', 'la', 'e', 'e', 'e'), 20), (('sa', 'e', 'le', 'a'), 21), (('e', 'a', 'la', 'e', 'sa'), 21), (('e', 'sa', 'la', 'pa', 'a'), 21), (('pa', 'a', 'sa', 'a', 'a'), 20), (('sa', 'a', 'pa', 'a'), 20), (('a', 'i', 'la', 'le'), 20), (('la', 'la', 'a', 'la', 'a', 'a'), 20), (('la', 'la', 'a', 'i', 'a'), 21), (('e', 'e', 'la', 'a', 'a', 'a', 'e'), 20), (('e',

In [113]:
print(freq_seqs_1)

{(('la', 'a', 'la', 'la', 'la', 'la', 'a'), 20), (('a', 'a', 'le', 'la', 'a'), 21), (('a', 'e', 'a', 'e', 'le', 'e'), 21), (('e', 'e', 'a', 'e', 'le'), 21), (('e', 'la', 'a', 'e', 'a', 'a', 'a', 'a'), 20), (('a', 'e', 'le', 'la', 'a'), 21), (('a', 'sa', 'i', 'le', 'la'), 20), (('e', 'a', 'sa', 'a', 'le', 'a'), 20), (('lə', 'a', 'a'), 20), (('e', 'sa', 'la', 'sa'), 21), (('la', 'e', 'a', 'le'), 21), (('pa', 'a', 'sa', 'la', 'le', 'a'), 20), (('a', 'la', 'e', 'la', 'i'), 20), (('e', 'e', 'e', 'sa', 'le', 'a'), 20), (('e', 'pa', 'a', 'sa', 'la', 'la'), 20), (('a', 'a', 'e', 'la', 'la', 'a'), 20), (('pa', 'la', 'a', 'la', 'le'), 20), (('a', 'sa', 'la', 'e', 'e', 'e'), 20), (('sa', 'e', 'le', 'a'), 21), (('e', 'a', 'la', 'e', 'sa'), 21), (('e', 'sa', 'la', 'pa', 'a'), 21), (('pa', 'a', 'sa', 'a', 'a'), 20), (('sa', 'a', 'pa', 'a'), 20), (('a', 'i', 'la', 'le'), 20), (('la', 'la', 'a', 'la', 'a', 'a'), 20), (('la', 'la', 'a', 'i', 'a'), 21), (('e', 'e', 'la', 'a', 'a', 'a', 'e'), 20), (('e',